In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

In [2]:
epochs = 10
batch_size = 200
learning_rate = 0.001

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=True,
                    transform=transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081,))])),
    batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=False,
                    transform=transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081,))])),
    batch_size = batch_size,shuffle = True)

In [7]:
for idx,(train,target) in enumerate(train_loader):
    print(idx)
    print(train.shape)
    print(target)
    break

0
torch.Size([200, 1, 28, 28])
tensor([ 9,  3,  3,  0,  9,  6,  9,  8,  1,  4,  0,  7,  3,  2,
         9,  2,  9,  6,  9,  3,  1,  4,  4,  7,  3,  0,  2,  4,
         9,  8,  2,  7,  9,  4,  0,  9,  6,  2,  3,  4,  8,  4,
         6,  7,  6,  5,  2,  7,  7,  7,  0,  0,  1,  8,  9,  0,
         0,  5,  3,  9,  1,  0,  9,  0,  9,  4,  4,  0,  4,  3,
         0,  3,  1,  5,  0,  6,  1,  5,  7,  1,  8,  6,  6,  4,
         0,  2,  2,  3,  3,  4,  0,  6,  6,  4,  4,  2,  6,  1,
         6,  4,  1,  7,  3,  8,  9,  2,  5,  7,  0,  7,  3,  2,
         1,  6,  6,  5,  5,  6,  4,  4,  8,  9,  2,  7,  2,  8,
         9,  8,  3,  8,  4,  2,  6,  2,  3,  8,  7,  7,  1,  7,
         0,  4,  2,  2,  9,  8,  9,  2,  6,  9,  2,  6,  3,  7,
         0,  1,  4,  5,  4,  9,  3,  5,  2,  1,  7,  5,  2,  7,
         5,  4,  8,  6,  7,  0,  5,  0,  5,  6,  6,  0,  7,  4,
         4,  5,  4,  4,  5,  1,  9,  6,  4,  7,  1,  3,  2,  6,
         2,  3,  3,  6])


In [9]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784,200),
            nn.ReLU(inplace = True),
            nn.Linear(200,200),
            nn.ReLU(inplace = True),
            nn.Linear(200,10),
            nn.ReLU(inplace=True)
        )
    
    def forward(self,x):
        x = self.model(x)
        return x

In [18]:
model = MLP()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    for idx,(train,target) in enumerate(train_loader):
        train = train.view(-1,28*28)#torch.Size([200, 1, 28, 28])
        logits = model(train)#torch.Size([200, 10])
        loss = criterion(logits,target)
        #print(loss.type())torch.FloatTensor
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
#         if idx%100 == 0:
#             print('Train epoch: {} [{}/{} ({:.0f}%)] \t Loss: {:.6f}'
#                       .format(epoch,idx*len(train),len(train_loader.dataset),100. * idx / len(train_loader),loss.item()))
    test_loss = 0
    correct = 0
    for data,tar in test_loader:
        data = data.view(-1,28*28)#
        logs = model(data)
        test_loss += criterion(logs,tar).item()
        pre_data = logs.data.max(1)[1]#torch.Size([200, 10])
        print(logs.data.max(1)[0])
        print(logs.data.max(1)[1])
        correct += pre_data.eq(tar).sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

tensor([  8.0283,   2.9947,   7.2803,  11.3934,  10.1908,   9.0027,
          4.9886,   0.0000,  11.1896,   6.1969,   8.8827,   6.0346,
          1.6992,   8.6076,   0.0000,   9.2996,   7.4054,   6.5572,
          0.0000,   4.4171,   6.8741,   9.8785,   6.2014,   6.9179,
          0.0000,   0.7736,   5.2885,   5.1085,   7.3494,   7.9640,
          3.4683,   9.8528,  11.9251,   6.5754,   0.7580,   6.3825,
          9.6941,   0.0000,   3.5718,   0.0000,   6.1257,   2.8271,
          4.0614,   0.0000,  12.2049,   6.1494,   9.8572,   7.9559,
          0.0000,   7.1491,   7.4322,   7.1322,   6.7709,   5.0640,
          1.0072,   4.7917,   5.2630,   5.9734,  13.6068,   9.3586,
         13.1861,   9.0020,   8.8005,   6.0497,   9.5449,  10.5247,
          7.2888,   6.8119,   7.1597,   6.1879,   8.5015,   0.0000,
          4.2134,   0.0000,   8.5744,   5.1043,   0.0000,   5.7469,
          9.3757,   9.9559,   4.0903,   0.0000,   7.1782,   9.5431,
          8.1841,   0.0000,   8.5303,   0.0000, 

tensor([  8.1000,  11.3366,   8.6125,   6.7237,   2.1609,   5.2541,
          0.3285,   8.8995,   0.0000,   5.4038,   0.0000,   8.7930,
          8.3211,   5.2745,   2.3944,   0.0000,   6.6343,   0.0000,
          0.3087,   0.0000,   0.0000,   9.5246,   5.5897,   8.3572,
          8.0631,   9.2473,  10.7466,   7.7814,   9.6877,   6.0939,
          7.5890,   0.0000,   7.3962,   3.4514,   0.0000,   9.7019,
         11.8191,   0.8528,   2.5579,   8.2107,   5.1885,   9.4532,
          5.8625,   8.7177,   2.3912,   5.4448,  11.4398,   6.9680,
          9.6560,  12.8580,   5.5989,   7.8979,  11.0545,  11.7226,
         10.8296,   8.3315,   0.0000,   0.0000,   5.4409,   8.4082,
          8.8150,   0.0000,   9.1017,   8.5738,   7.2054,   7.1549,
          9.6935,   4.3216,   7.3605,   8.6412,   0.0000,   0.0000,
          7.8948,   0.0000,   0.0000,   4.7182,   8.8896,   4.3556,
          0.0000,   7.3198,   8.2207,   6.2799,   8.6888,   7.4020,
         10.3388,   0.0000,   9.2100,   6.3981, 

tensor([  7.6788,   6.9453,   7.5919,   0.0000,   0.7296,   7.6529,
          9.5332,   3.9977,   8.2383,   1.6194,  11.0448,   8.1214,
          2.5668,   9.5394,   6.7449,   9.7353,   0.0000,   7.0746,
          6.4677,   5.2774,   9.2649,  13.3223,   8.3457,   4.7825,
          6.4794,   6.0522,   7.5828,   9.3754,   6.8419,  10.3132,
         10.2367,   0.0000,   9.7489,   8.1082,   0.0000,   0.0000,
          7.7280,   0.1447,   7.1230,   6.4520,   2.6004,   0.0381,
          7.2198,  13.3811,   5.5448,   8.1698,  11.1245,   7.3805,
         10.1042,   2.5957,   0.0000,   8.0770,   9.7762,   9.1760,
          8.5157,   9.1864,   6.2537,   0.8040,  10.3480,   9.7838,
          4.5022,   5.4861,  13.9024,   8.7896,   4.3807,   7.9664,
          0.0000,   5.1749,   9.0887,   9.3043,   0.1758,   6.1746,
         11.4266,   7.2106,  10.7878,   1.7084,   6.9794,   6.5507,
          7.1950,   3.6290,   6.7594,   8.3449,   0.0000,   5.9692,
          7.0007,   7.3793,  11.2432,   0.0000, 

tensor([ 4,  5,  6,  0,  3,  9,  0,  0,  5,  0,  2,  1,  1,  0,
         5,  1,  4,  3,  3,  1,  1,  9,  6,  2,  4,  4,  9,  1,
         1,  6,  5,  1,  6,  0,  6,  0,  9,  1,  6,  4,  6,  4,
         0,  3,  4,  6,  1,  4,  5,  0,  0,  0,  2,  1,  1,  6,
         1,  1,  0,  1,  0,  1,  4,  3,  2,  0,  5,  6,  1,  0,
         4,  6,  0,  1,  3,  9,  9,  4,  4,  3,  6,  5,  1,  3,
         1,  1,  0,  5,  9,  0,  2,  4,  3,  4,  9,  5,  1,  1,
         2,  4,  0,  2,  1,  1,  1,  1,  5,  0,  1,  3,  0,  1,
         2,  2,  5,  4,  2,  0,  3,  5,  9,  1,  0,  2,  6,  0,
         2,  9,  3,  9,  4,  0,  6,  4,  0,  4,  4,  6,  1,  4,
         3,  5,  2,  2,  4,  6,  9,  3,  0,  5,  2,  0,  2,  1,
         0,  5,  2,  1,  1,  1,  6,  0,  5,  4,  1,  6,  9,  5,
         5,  0,  2,  3,  4,  1,  9,  9,  0,  5,  1,  2,  5,  6,
         3,  2,  2,  0,  6,  3,  3,  5,  0,  4,  1,  3,  9,  5,
         5,  0,  9,  3])
tensor([  4.7229,   4.4124,   8.5182,   5.9055,   6.0993,   5.4568,
          0

tensor([  0.0000,   8.5173,   8.3856,   2.3315,   8.5671,   6.9740,
         10.2761,   6.3072,  11.0114,   7.1422,   9.3112,   7.9137,
          6.6671,   4.0530,   8.6139,   7.4727,   7.5014,  12.5253,
         11.3264,   7.3477,   8.5407,   0.0000,   8.1926,   6.5689,
         10.2953,   7.6267,   6.6664,   7.1881,   9.5958,   2.5369,
          2.2028,   0.0000,   9.5185,   9.8542,   6.7439,   3.9396,
         11.2151,   5.8005,   2.8224,   8.4551,  12.8037,   7.5343,
          0.0000,   4.3028,   0.0000,  12.7191,   9.7398,   7.5665,
          2.0118,   9.0208,   8.4616,   6.0754,   0.0000,   2.6939,
          6.8508,   0.0000,   2.5661,   7.2933,   4.8042,   0.0000,
          8.0062,   4.3822,   5.1683,   5.6205,   5.3643,   9.5815,
          7.2179,   0.0000,   6.8389,   6.2064,   8.9530,   8.2662,
          4.7867,   5.6657,   9.1113,   5.5657,  10.5766,   8.6536,
          3.2286,   8.6263,   7.5227,   3.9587,   9.1332,   0.3229,
          9.3612,  10.0719,   7.3948,   8.6644, 

tensor([ 12.3297,   0.0000,   3.9157,   8.2539,   9.0976,  10.4532,
          7.9518,   7.7060,   8.3610,   0.0000,   7.2943,   0.0000,
          8.2624,   5.3109,  11.4266,  14.5986,   6.5916,   8.9874,
          8.2856,   9.3310,   5.1255,   7.5617,   8.1392,  10.8423,
         10.6889,   9.8290,   6.0043,   7.3780,   7.0956,  10.2064,
         11.6086,   0.0000,  12.3286,  11.0149,   0.0000,  11.4090,
          8.6675,  11.8282,   7.7451,  10.2805,   2.4670,  11.1760,
          8.6753,   8.3060,   8.6872,   9.1071,   0.0000,   5.6696,
          8.8822,   9.7681,   4.5337,   8.6634,   0.0000,   7.9332,
          9.4474,   6.2292,  10.1654,   0.0000,   3.7039,   5.2414,
          9.7542,   5.5884,   7.9673,   6.8708,  11.4444,   6.3182,
          7.9842,   5.2460,   9.1536,   9.2014,   7.7689,   7.2108,
         10.9555,   8.5348,   8.7749,   4.4509,  14.2462,   9.8580,
          8.9190,   1.4393,   0.0000,  10.6827,   9.5767,   9.9141,
         10.1778,   3.9473,   8.2534,   7.6497, 

KeyboardInterrupt: 